# Cross Entropy Method

How do we solve  for the policy optimization problem which is to **maximize** the total reward given some parametrized policy? 

## Discounted future reward

To begin with, for an episode the total reward is the sum of all the rewards. If our environment is stochastic, we can never be sure if we will get the same rewards the next time we perform the same actions. Thus the more we go into the future the more the total future reward may diverge. So for that reason it is common to use the **discounted future reward** where the parameter `discount` is called the discount factor and is between 0 and 1. 

A good strategy for an agent would be to always choose an action that maximizes the (discounted) future reward. In other words we want to maximize the expected reward per episode.

## Parametrized policy

A stochastic policy is defined as a conditional probability of some action given a state. A family of policies indexed by a parameter vector `theta` are called parametrized policies. These policies are defined analogous to the supervised learning classification or regression problems. In the case of discrete policies we output a vector of probabilities of the possible actions and in the case of continuous policies we output a mean and diagonal covariance of a Gaussian distribution from which we can then sample our continous actions.

## Cross entropy method (CEM)

So how do we solve for the policy optimization problem of maximizing the total (discounted) reward given some parametrized policy? The simplest approach is the derivative free optimization (DFO) which looks at this problem as a black box with respect to the parameter `theta`. We try out many different `theta` and store the rewards for each episode. The main idea then is to move towards good `theta`.

One particular DFO approach is called the CEM. Here at any point in time, you maintain a distribution over parameter vectors and move the distribution towards parameters with higher reward. This works surprisingly well, even if its not that effictive when `theta` is a high dimensional vector.

## Algorithm

The idea is to initialize the `mean` and `sigma` of a Gaussian and then for `n_iter` times we:

1. collect `batch_size` samples of `theta` from a Gaussian with the current `mean` and `sigma`
2. perform a noisy evaluation to get the total rewards with these `theta`s 
3. select `n_elite` of the best `theta`s into an elite set
4. upate our `mean` and `sigma` to be that from the elite set

## Discrete linear policy

For the `CartPole-v0` case let us define the linear parametrized policy as the following diagram:

```
         │               ┌───theta ~ N(mean,std)───┐
         │
   4 observations        [[ 2.2  4.5 ]
[-0.1 -0.4  0.06  0.5] *  [ 3.4  0.2 ]  + [[ 0.2 ]
         |                [ 4.2  3.4 ]     [ 1.1 ]]
         │                [ 0.1  9.0 ]]
         |                     W              b
    ┌────o────┐
<─0─│2 actions│─1─>    = [-0.4  0.1] ──argmax()─> 1
    └─o─────o─┘
```

Which means we can use the `Space` introspection of the `env` to create an appropriatly sized `theta` parameter vector from which we can use a part as the matrix `W` and the rest as the bias vector `b` so that the number of output probabilities correspond to the number of actions of our particular `env`.

## Extra noise

We can also add extra decayed noise to our distribution in the form of `extra_cov` which decays after `extra_decay_time` iterations.

## Discounted total reward

We can also return the discounted total reward per episode via the `discount` parameter of the `do_episode` function:

```python
...
for t in xrange(num_steps):
  ...
  disc_total_rew += reward * discount**t
  ...
```

In [1]:
import numpy as np
import gym

from keras.models import Input, Model
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

from rl.agents.cem import CEMAgent
from rl.memory import EpisodeParameterMemory

Using TensorFlow backend.


In [2]:
ENV_NAME = 'CartPole-v0'

Get the environment and extract the number of actions.

In [3]:
env = gym.make(ENV_NAME)
np.random.seed(123)
env.seed(123)
nb_actions = env.action_space.n
obs_dim = env.observation_space.shape[0]

Option 1 : Simple model

In [4]:
inp = Input(shape=(1,) + env.observation_space.shape)
x = Flatten()(inp)
x = Dense(nb_actions)(x)
x = Activation('softmax')(x)
model = Model(inputs=inp, outputs=x)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1, 4)              0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 10        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


Option 2: deep network

In [5]:
inp = Input(shape=(1,) + env.observation_space.shape)
x = Flatten()(inp)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(16)(x)
x = Activation('relu')(x)
x = Dense(nb_actions)(x)
x = Activation('softmax')(x)
model = Model(inputs=inp, outputs=x)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 4)              0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
__________

Finally, we configure and compile our agent. You can use every built-in Keras optimizer and even the metrics!

In [6]:
memory = EpisodeParameterMemory(limit=1000, window_length=1)
cem = CEMAgent(model=model, nb_actions=nb_actions, memory=memory,
               batch_size=50, nb_steps_warmup=2000, train_interval=50, elite_frac=0.05)
cem.compile()

Okay, now it's time to learn something! We visualize the training here for show, but this slows down training quite a lot. You can always safely abort the training prematurely using Ctrl + C. cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

In [11]:
cem.fit(env, nb_steps=1000, visualize=False, verbose=2)
#cem.fit(env, nb_steps=100000, visualize=False, verbose=2)

Training for 1000 steps ...
  21/1000: episode: 1, duration: 0.116s, episode steps: 21, steps per second: 180, episode reward: 21.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.381 [0.000, 1.000], mean observation: 0.060 [-1.001, 1.740], mean_best_reward: --
  45/1000: episode: 2, duration: 0.014s, episode steps: 24, steps per second: 1736, episode reward: 24.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.417 [0.000, 1.000], mean observation: 0.127 [-0.944, 1.877], mean_best_reward: --
  60/1000: episode: 3, duration: 0.011s, episode steps: 15, steps per second: 1359, episode reward: 15.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.667 [0.000, 1.000], mean observation: -0.073 [-1.970, 1.230], mean_best_reward: --
  71/1000: episode: 4, duration: 0.009s, episode steps: 11, steps per second: 1194, episode reward: 11.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.182 [0.000, 1.000], mean observation: 0.116 [-1.397, 2.283], mean_best_reward: --
  84/1000: e

 753/1000: episode: 41, duration: 0.031s, episode steps: 46, steps per second: 1474, episode reward: 46.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.565 [0.000, 1.000], mean observation: 0.127 [-1.527, 1.528], mean_best_reward: --
 765/1000: episode: 42, duration: 0.014s, episode steps: 12, steps per second: 865, episode reward: 12.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.250 [0.000, 1.000], mean observation: 0.093 [-1.227, 1.992], mean_best_reward: --
 774/1000: episode: 43, duration: 0.011s, episode steps: 9, steps per second: 827, episode reward: 9.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.111 [0.000, 1.000], mean observation: 0.176 [-1.337, 2.359], mean_best_reward: --
 784/1000: episode: 44, duration: 0.018s, episode steps: 10, steps per second: 565, episode reward: 10.000, mean reward: 1.000 [1.000, 1.000], mean action: 0.100 [0.000, 1.000], mean observation: 0.139 [-1.526, 2.479], mean_best_reward: --
 799/1000: episode: 45, duration: 0.025s,

After training is done, we save the best weights.

In [12]:
cem.save_weights('cem_{}_params.h5f'.format(ENV_NAME), overwrite=True)

Finally, evaluate our algorithm for 5 episodes.

In [14]:
cem.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 10.000, steps: 10
Episode 2: reward: 10.000, steps: 10
Episode 3: reward: 10.000, steps: 10
Episode 4: reward: 10.000, steps: 10
Episode 5: reward: 9.000, steps: 9
